In [ ]:
from flask import Flask, request, jsonify
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoFeatureExtractor, AutoModelForImageClassification
from PIL import Image
import torch


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from diffusers import StableDiffusionPipeline

from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from PIL import Image
import torch

In [ ]:
# Load the image captioning model ( load model directly )
caption_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
caption_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [ ]:
# Load the emotion detection model
emotion_extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-50")
emotion_model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-50")

In [ ]:
# Function to generate an emotional caption for an image
def generate_emotional_caption(image_path):
    try:
        image = Image.open(image_path)

        # Generate the base caption
        inputs = caption_processor(images=image, return_tensors="pt")
        outputs = caption_model.generate(**inputs)
        base_caption = caption_processor.decode(outputs[0], skip_special_tokens=True)

        # Detect emotion
        inputs = emotion_extractor(images=image, return_tensors="pt")
        with torch.no_grad():
            logits = emotion_model(**inputs).logits
        predicted_class = logits.argmax(-1).item()
        emotion = emotion_model.config.id2label[predicted_class]

    

        # Enhance the caption with emotion
        emotional_caption = f"{base_caption}. The scene evokes a sense of {emotion.lower()}."
        return emotional_caption
    except Exception as e:
        return f"Error generating caption: {str(e)}"

In [ ]:
import json
from flask import Flask, jsonify, request
from flask_cors import CORS

In [ ]:


app = Flask(__name__)
CORS(app)
@app.route('/upload', methods=['POST'])
def uploadimg():
    file = request.files['image']
    file_path = os.path.join('uploads', file.filename)
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    file.save(file_path)
    caption = generate_emotional_caption(file_path)
    return jsonify({'caption': caption})

if __name__ == '__main__':
    app.run(port=5000)

In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import torch

In [ ]:
# Example usage
image_path = "Image.jpg"
print(generate_emotional_caption(image_path))